In [1]:
!nvidia-smi

Fri Sep 15 08:38:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/anilbhatt1/ERA1_S17_BERT_GPT_VIT.git

Cloning into 'ERA1_S17_BERT_GPT_VIT'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 59 (delta 18), reused 51 (delta 10), pack-reused 0
Receiving objects: 100% (59/59), 4.47 MiB | 25.45 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [3]:
import os
print(os.getcwd())
os.chdir('/content/ERA1_S17_BERT_GPT_VIT/assignment')
print(os.getcwd())

/content
/content/ERA1_S17_BERT_GPT_VIT/assignment


In [4]:
!pip install -r requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.8 MB/s eta 0:00:00


In [5]:
import torch
from transformer import *
from transformers import AutoTokenizer
import yaml
from gpt_utils import *

In [6]:
# Load the YAML file
with open('/content/ERA1_S17_BERT_GPT_VIT/assignment/config.yaml', 'r') as yaml_file:
    cfg = yaml.load(yaml_file, Loader=yaml.FullLoader)
cfg['model'] = 'gpt'
model_nm = cfg['model']
print(model_nm)
cfg

gpt


{'model': 'gpt',
 'bert': {'batch_size': 1024,
  'seq_len': 20,
  'embed_size': 128,
  'inner_ff_size': 512,
  'n_heads': 8,
  'n_code': 8,
  'n_vocab': 40000,
  'dropout': 0.1,
  'n_workers': 12,
  'optim_kwargs': {'lr': 0.0001,
   'weight_decay': 0.0001,
   'betas': [0.9, 0.999]},
  'sentence_path': '/content/ERA1_S17_BERT_GPT_VIT/course_docs/BERT/training.txt',
  'vocab_path': '/content/ERA1_S17_BERT_GPT_VIT/course_docs/BERT/vocab.txt',
  'data_kwargs': {'shuffle': True,
   'drop_last': False,
   'pin_memory': True,
   'batch_size': 1024},
  'train_iter': 2000,
  'print_iter': 10,
  'embed_save_limit': 3000},
 'gpt': {'batch_size': 32,
  'block_size': 64,
  'dropout': 0.2,
  'lr': 0.0003,
  'num_embed': 768,
  'num_head': 6,
  'num_layer': 6,
  'max_iter': 1000,
  'eval_inter': 200,
  'data_path': '/content/ERA1_S17_BERT_GPT_VIT/course_docs/GPT/data/english.txt',
  'inference_tokens': 50}}

In [7]:
print('initializing from config data..')
BATCH_SIZE = cfg[model_nm]['batch_size']
BLOCK_SIZE = cfg[model_nm]['block_size']
DROPOUT = cfg[model_nm]['dropout']
LEARNING_RATE = cfg[model_nm]['lr']
NUM_EMBED = cfg[model_nm]['num_embed']
NUM_HEAD = cfg[model_nm]['num_head']
NUM_LAYER = cfg[model_nm]['num_layer']
MAX_ITER = cfg[model_nm]['max_iter']
EVAL_INTER = cfg[model_nm]['eval_inter']
path_to_data = cfg[model_nm]['data_path']
inference_tokens = cfg[model_nm]['inference_tokens']
inference_tokens

initializing from config data..


50

In [8]:
# raw data
data_raw = open(path_to_data, encoding="utf-8").read()
# we use pretrained BERT tokenizer for performance improvements
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
vocab_size = tokenizer.vocab_size
# data_raw = data_raw[4000000:] # short dataset

# train/val split
data = encode(text_seq=data_raw, tokenizer=tokenizer)
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

Token indices sequence length is longer than the specified maximum sequence length for this model (37443 > 512). Running this sequence through the model will result in indexing errors


In [9]:
# train a new model
model = gpt_Transformer(
    vocab_size=vocab_size,
    num_embed=NUM_EMBED,
    block_size=BLOCK_SIZE,
    num_heads=NUM_HEAD,
    num_layers=NUM_LAYER,
    dropout=DROPOUT,
)
# load model to GPU if available
m = model.to(DEVICE)
# print the number of parameters in the model
print(
    "Model with {:.2f}M parameters".format(sum(p.numel() for p in m.parameters()) / 1e6)
)

Model with 89.48M parameters


In [10]:
# optimizer takes the model's parameters and the learning rate as input,
# and updates the parameters during the training process in order to
# minimize the loss function.
optimizer = torch.optim.AdamW(m.parameters(), lr=LEARNING_RATE)
for step in range(MAX_ITER):

    # every EVAL_INTER evaluate the loss on train and val sets
    if step % EVAL_INTER == 0 or step == MAX_ITER - 1:
        loss_train = estimate_loss(
            data=train_data, model=m, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE
        )
        loss_val = estimate_loss(
            data=val_data, model=m, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE
        )
        print("step {:10} | train loss {:6.4f} | val loss {:6.4f}".format(step, loss_train, loss_val))

    # sample a batch of data
    xb, yb = get_batch(data=train_data, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE)
    logits, loss = m.forward(xb, yb)
    # zero_grad() method sets the gradients of all parameters in the optimizer to zero
    optimizer.zero_grad(set_to_none=True)
    # backward() method on the loss variable calculates the gradients
    # of the loss with respect to the model's parameters.
    loss.backward()
    # step() method on the optimizer updates the model's parameters
    # using the calculated gradients, in order to minimize the loss.
    optimizer.step()
    print(f'step : {step}, loss : {loss.item()}')

step          0 | train loss 10.7351 | val loss 10.7551
step : 0, loss : 10.765129089355469
step : 1, loss : 9.729708671569824
step : 2, loss : 9.858390808105469
step : 3, loss : 9.353569030761719
step : 4, loss : 9.240790367126465
step : 5, loss : 8.986915588378906
step : 6, loss : 8.532050132751465
step : 7, loss : 8.109743118286133
step : 8, loss : 7.850605487823486
step : 9, loss : 7.2997541427612305
step : 10, loss : 6.998342037200928
step : 11, loss : 6.662104606628418
step : 12, loss : 6.85446834564209
step : 13, loss : 6.550704002380371
step : 14, loss : 6.442161560058594
step : 15, loss : 6.391232013702393
step : 16, loss : 6.187860488891602
step : 17, loss : 6.077338695526123
step : 18, loss : 6.359762668609619
step : 19, loss : 6.221409797668457
step : 20, loss : 5.853934288024902
step : 21, loss : 5.840162754058838
step : 22, loss : 5.818700790405273
step : 23, loss : 5.936500072479248
step : 24, loss : 5.994067192077637
step : 25, loss : 5.9601311683654785
step : 26, loss 

In [11]:
save_model_to_chekpoint(model=m, path_to_checkpoint="checkpoint", epoch=step)

Successfully saved the model to checkpoint/checkpoint_epoch-999_15.09.2023_08:47:58.pt


In [12]:
# generate some output based on the context
context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
print(
    decode(
        enc_sec=m.generate(idx=context, max_new_tokens=inference_tokens, block_size=BLOCK_SIZE)[0],
        tokenizer=tokenizer,
    )
)

[PAD] : normalized our kernel is stopped ( figuratively in the animation above ), we are looking / reading 9 pixels / reading 9 pixels ( we covered only 1 pixel 9 times ) we will have a resolution of 3x3 pixels. we multiply
